In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Here data is availble in form of images grouped based on the classes as a folder name.<br>
Let's first get path of training and test set images as numpy array and also get all training labels.

In [ ]:
train_dir = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
test_dir = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

from sklearn.datasets import load_files

def load_dataset(path):
    data = load_files(path) #load all files from the path
    files = np.array(data['filenames']) #get the file  
    targets = np.array(data['target'])#get the the classification labels as integer index
    target_labels = np.array(data['target_names'])#get the the classification labels 
    return files,targets,target_labels
    
x_train, y_train,target_labels = load_dataset(train_dir)
x_test, y_test,_ = load_dataset(test_dir)

In [ ]:
print('Training set size : ' , x_train.shape[0])
print('Testing set size : ', x_test.shape[0])
print('No of fruits for classification: ', len(target_labels))

Let's have a look at training data distribution.

In [ ]:
import matplotlib.pyplot as plt
unique,counts = np.unique(y_train, return_counts=True)#Get count for each fruit
index = np.arange(len(target_labels))

plt.figure(figsize=(15,30))
plt.barh(index, counts)
plt.ylabel('Fruits', fontsize=25)
plt.xlabel('Fruits count', fontsize=25)
plt.yticks(index, target_labels, fontsize=13)
plt.show()

Split training data as 80% training set and 20% validation set using scikit learn's train_test_split method.

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,random_state = 1)

In [ ]:
print ("x_train shape: " + str(x_train.shape))
print ("x_train shape: " + str(y_train.shape))
print ("x_validate shape: " + str(x_validate.shape))
print ("y_validate shape: " + str(y_validate.shape))
print ("x_test shape: " + str(x_test.shape))
print ("y_test shape: " + str(y_test.shape))

Let's convert images in numpy array.

In [ ]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert to Numpy Array
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array

x_train = np.array(convert_image_to_array(x_train))
print('Training set shape : ',x_train.shape)

x_validate = np.array(convert_image_to_array(x_validate))
print('Validation set shape : ',x_validate.shape)

x_test = np.array(convert_image_to_array(x_test))
print('Test set shape : ',x_test.shape)

Calling garbage collector to avoid memory error.

In [ ]:
import gc
gc.collect()

Rescale image data to (0,1) range from (0,255) range

In [ ]:
x_train = x_train.astype('float32')/255
x_validate = x_validate.astype('float32')/255
x_test = x_test.astype('float32')/255

Let's have alook at some of fruit images.

In [ ]:
plt.figure(figsize=(15,12))

for i in range(1,17):
    index = np.random.randint(x_train.shape[0])
    plt.subplot(4, 4, i)
    plt.imshow(np.squeeze(x_train[index]), cmap='cool')
plt.show()

Perform data augmentation for generalization of training data set

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images
datagen.fit(x_train)

In the following CNN model Convolutional layers and fully connected layers are taken from following paper:<br>
[Fruit recognition from images using deep learning](https://www.researchgate.net/publication/321475443_Fruit_recognition_from_images_using_deep_learning)<br>
Additon to that max pool layers, dropout layers and batch normalization layers are added.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Conv2D(16,kernel_size=(5, 5),kernel_initializer='he_normal',activation='relu',input_shape=(100,100,3),name = 'conv0'))
model.add(BatchNormalization(name='bn0'))
model.add(Dropout(0.2,name='dropout0'))

model.add(Conv2D(32, kernel_size=(5, 5), activation='relu',name = 'conv1'))
model.add(BatchNormalization(name='bn1'))
model.add(MaxPooling2D(pool_size=(2, 2),name = 'maxpool1'))
model.add(Dropout(0.2,name='dropout1'))

model.add(Conv2D(64, kernel_size=(5, 5), activation='relu',name = 'conv2'))
model.add(BatchNormalization(name='bn2'))
model.add(MaxPooling2D(pool_size=(2, 2),name = 'maxpool2'))
model.add(Dropout(0.2,name='dropout2'))

model.add(Conv2D(128, kernel_size=(5,5), activation='relu',name = 'conv3'))
model.add(BatchNormalization(name='bn3'))
model.add(MaxPooling2D(pool_size=(2, 2),name = 'maxpool3'))
model.add(Dropout(0.2,name='dropout3'))

model.add(Flatten(name='fc'))
model.add(Dense(1024, activation='relu',name = 'Dense0'))
model.add(Dense(256, activation='relu',name = 'Dense1'))
model.add(Dropout(0.3,name='dropout4'))
model.add(Dense(len(target_labels), activation='softmax',name = 'Dense2'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

Create checkpoint to save the best model

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath = 'cnn_fruit_360.hdf5', verbose = 1, save_best_only = True)

In [ ]:
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size= 32), epochs = 50, verbose=1,callbacks = [checkpoint],validation_data=(x_validate,y_validate))

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validate'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validate'], loc='upper left')
plt.show()

Load weights from the best saved model to evaluate test set.

In [ ]:
model.load_weights('cnn_fruit_360.hdf5')

In [ ]:
score = model.evaluate(x_test,y_test,verbose=0)
print('Test Loss :',score[0])
print('Test Accuracy :',score[1])

In [ ]:
#get the predictions for the test data
predicted_classes = model.predict_classes(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, predicted_classes, target_names = target_labels))

In [ ]:
confusion_mtx = confusion_matrix(y_true, predicted_classes) 
import itertools
plt.imshow(confusion_mtx, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('confusion_matrix')
plt.colorbar()
tick_marks = np.arange(len(target_labels))
plt.xticks(tick_marks, target_labels, rotation=90)
plt.yticks(tick_marks, target_labels)
#Following is to mention the predicated numbers in the plot and highligh the numbers the most predicted number for particular label
thresh = confusion_mtx.max() / 2.
for i, j in itertools.product(range(confusion_mtx.shape[0]), range(confusion_mtx.shape[1])):
    plt.text(j, i, confusion_mtx[i, j],
    horizontalalignment="center",
    color="white" if confusion_mtx[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')